# 11 — Errors, Exceptions & Debugging Patterns

Goal: Get comfortable with how Python fails, how to read error messages, and how to handle exceptions without turning your code into a try/except soup.

In practice this means:

- Recognising common error types (SyntaxError, TypeError, ValueError, IndexError, KeyError, etc.)
- Reading tracebacks so you know **where** and **why** things blew up
- Using `try` / `except` / `finally` / `raise` in a controlled way
- Adopting a few simple debugging habits (print-style debugging, small repros, isolating functions)

The aim isn’t to avoid errors — it’s to **use them as information** instead of getting stuck or randomly poking at the code.
